In [1]:
import glob
import json
import pandas as pd
import numpy as np

import urllib.request as urllib2

from IPython.display import JSON

# Functions

In [ ]:
# TODO - Turn into 1 liner
def facility2CMSNum (facilityName):
    if facilityName in ltc_name2cms_id:
        return ltc_name2cms_id[facilityName]
    else:
        return "No Match"

#  To be refactored
def get_matching_cms_date(date2Match):
    dates = list(state_cms_data['CMS_Week Ending'].unique())
    dates.sort()
    for date in dates:
        matchDate = date
        if np.datetime64(date2Match) <= date :
           break
    #     matchDate = date
    return matchDate


def add_cms_data_to_df(df, cms_df, df_key, cms_key):
    ''' 
        Steo 1 - Add CMS Provider Number based on Facility Name field
        Step 2 - Merge with CMS data
    '''
    # Specify columns from CMS data feed to add
    cols=['CMS_Week Ending', 'CMS_Federal Provider Number', 'CMS_Provider Name',
           'CMS_Provider Address', 'CMS_Provider City', 'CMS_Provider State', 'CMS_County', 
           'CMS_Provider Zip Code', 'CMS_Submitted Data',
           'CMS_Residents Weekly Admissions COVID-19',
           'CMS_Residents Total Admissions COVID-19',
           'CMS_Residents Weekly Confirmed COVID-19',
           'CMS_Residents Total Confirmed COVID-19',
           'CMS_Residents Weekly Suspected COVID-19',
           'CMS_Residents Total Suspected COVID-19', 
           'CMS_Residents Weekly All Deaths',
           'CMS_Residents Total All Deaths', 
           'CMS_Residents Weekly COVID-19 Deaths',
           'CMS_Residents Total COVID-19 Deaths', 'CMS_Number of All Beds',
           'CMS_Total Number of Occupied Beds',
           'CMS_Staff Weekly Confirmed COVID-19', 'CMS_Staff Total Confirmed COVID-19',
           'CMS_Staff Weekly Suspected COVID-19', 'CMS_Staff Total Suspected COVID-19',
           'CMS_Staff Weekly COVID-19 Deaths', 'CMS_Staff Total COVID-19 Deaths',
           'CMS_Shortage of Nursing Staff', 'CMS_Shortage of Clinical Staff',
           'CMS_Shortage of Aides', 'CMS_Shortage of Other Staff',
           'CMS_Weekly Resident Confirmed COVID-19 Cases Per 1,000 Residents',
           'CMS_Weekly Resident COVID-19 Deaths Per 1,000 Residents',
           'CMS_Total Resident Confirmed COVID-19 Cases Per 1,000 Residents',
           'CMS_Total Resident COVID-19 Deaths Per 1,000 Residents',
           'CMS_Total Residents COVID-19 Deaths as a Percentage of Confirmed COVID-19 Cases',
           'CMS_Three or More Confirmed COVID-19 Cases This Week',
           'CMS_Initial Confirmed COVID-19 Case This Week',
           'CMS_Resident Access to Testing in Facility',
#            'CMS_Able to Test or Obtain Resources to Test All Current Residents Within Next 7 Days',
#            'CMS_During Past Two Weeks Average Time to Receive Resident Test Results',
#            'CMS_Has Facility Performed Resident Tests Since Last Report',
#            'CMS_Tested Residents with New Signs or Symptoms',
#            'CMS_Tested Asymptomatic Residents in a Unit or Section After a New Case',
#            'CMS_Tested Asymptomatic Residents Facility-Wide After a New Case',
#            'CMS_Tested Asymptomatic Residents Without Known Exposure as Surveillance',
#            'CMS_Tested Another Subgroup of Residents',
           'CMS_Able to Test or Obtain Resources to Test All Staff and/or Personnel Within Next 7 Days',
           'CMS_During Past Two Weeks Average Time to Receive Staff and/or Personnel Test Results',
#            'CMS_Has Facility Performed Staff and/or Personnel Tests Since Last Report',
           'CMS_Tested Staff and/or Personnel with New Signs or Symptoms',
           'CMS_Tested Asymptomatic Staff and/or Personnel in a Unit or Section After a New Case',
           'CMS_Tested Asymptomatic Staff and/or Personnel Facility-Wide After a New Case',
           'CMS_Tested Asymptomatic Staff and/or Personnel Without Known Exposure as Surveillance',
#            'CMS_Tested Another Subgroup of Staff and/or Personnel',
           'CMS_In-House Point-of-Care Test Machine',
#            'CMS_COVID-19 Point-of-Care Tests Performed on Residents Since Last Report',
#            'CMS_COVID-19 Point-of-Care Tests Performed on Staff and/or Personnel Since Last Report',
#            'CMS_Enough Supplies to Test All Staff and/or Personnel Using Point-of-Care Test Machine',
#            'CMS_Any Current Supply of N95 Masks', 'CMS_One-Week Supply of N95 Masks',
#            'CMS_Any Current Supply of Surgical Masks',
#            'CMS_One-Week Supply of Surgical Masks',
#            'CMS_Any Current Supply of Eye Protection',
#            'CMS_One-Week Supply of Eye Protection', 'CMS_Any Current Supply of Gowns',
#            'CMS_One-Week Supply of Gowns', 'CMS_Any Current Supply of Gloves',
#            'CMS_One-Week Supply of Gloves', 'CMS_Any Current Supply of Hand Sanitizer',
#            'CMS_One-Week Supply of Hand Sanitizer', 
#            'CMS_Ventilator Dependent Unit',
#            'CMS_Number of Ventilators in Facility',
           'CMS_Number of Ventilators in Use for COVID-19',
#            'CMS_Any Current Supply of Ventilator Supplies',
#            'CMS_One-Week Supply of Ventilator Supplies',
           'CMS_Geolocation'
           ]
    df_w_cms = pd.merge(df, cms_df[cols], left_on=df_key, right_on=cms_key, how='left')
#     df_w_cms = pd.merge(df, cms_df, left_on=df_key, right_on=cms_key, how='left')

    return df_w_cms

In [3]:
def outbreak_df_from_file(filename):
    """ From Json file:
        1) return DataFrame augmented and save to file
        2) return Summary data"""
    with open(filename) as f:
      ltc_data = json.load(f)

    # Extract Reporting Data
    reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])
    df = pd.DataFrame(ltc_data['FacilityValues'])
    df.insert(0, 'reporting_date', reporting_date)
    df['CFR'] = (df['deaths'] / df['confirmed_cases'])
    df['outbreaks'] = 1 # to allow counting # of outbreaks by Facility
    #Save Outbreak data to a file
    outbreak_file = 'Reporting_data/IL_' + reporting_date + '_Outbreaks_LTC_data_v2.csv'
    df.to_csv(outbreak_file, index = False)
    
    # Get summary data from feed - Note this may not match totals - ST-TODO: Check if summary data and totals from raw data match
    deaths = ltc_data['LTC_Reported_Cases']['deaths']
    confirmed_cases = ltc_data['LTC_Reported_Cases']['confirmed_cases']
    facility_cnt = len(df.groupby(['County', 'FacilityName']).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=False))
    summary = {}
    summary['Date'] = reporting_date
    summary['Cases'] = confirmed_cases
    summary['Deaths'] = deaths
    summary['Outbreaks'] = df.reporting_date.value_counts()[0]
    summary['Facilities'] = facility_cnt
    
    return df, summary, reporting_date

In [4]:
def process_json_IL (filename, display_dfs=False, display_summary=True):
    """Process a JSON file to:
       1) Produce Summary Info
       2) Produce Outbreak file and dataframe
       3) Produce Facility file and dataframe
       4) Produce County file and dataframe
        
       TODO - make display dataframes optional
       TODO - make display summary info optional"""
    [outbreak_df, summary, reporting_date] = outbreak_df_from_file(filename)

    # Print Summary Data
    if display_summary:
        for k,v in summary.items():
            print(k + ": " + str(v))    

    # Save and Display Facility data
    df_facilities = outbreak_df.groupby(['County', 'FacilityName']).sum()
    df_facilities['CFR'] = df_facilities['deaths'] / df_facilities['confirmed_cases']
    df_facilities['facilities'] = 1
    df_facilities.insert(0, 'ReportingDate', reporting_date)
    df_facilities.sort_values(by='confirmed_cases', ascending=False).to_csv('Reporting_data/IL_' + reporting_date + '_Facilities_LTC_data_v2.csv')
    df_facilities.reset_index(inplace=True) 

    # Save and Display County Level Data
    df_county = df_facilities.groupby(by=['County']).sum()
    df_county['CFR'] = (df_county['deaths'] / df_county['confirmed_cases'])
    df_county.insert(0, 'ReportingDate', reporting_date)
    filename = 'Reporting_data/IL_' + reporting_date + '_County_LTC_stats_v2.csv'
    df_county.sort_values('confirmed_cases', ascending=False).to_csv('Reporting_data/IL_' + reporting_date + '_County_LTC_stats_v2.csv')
    df_county.reset_index(inplace=True)
    
    if display_dfs:
        print("\nOutbreak Data\n=============")
        display(outbreak_df.sort_values(by='deaths', ascending=False).head(5))
        print("\nFacility Data\n=============")
        display(df_facilities.sort_values('deaths', ascending=False).head(10))
        print("\nCounty Data\n===========")
        display(df_county.sort_values(by='confirmed_cases', ascending=False).head(10))

    return reporting_date, summary, outbreak_df.add_prefix('STATE_'), df_facilities.add_prefix('STATE_'), df_county

In [5]:
def facility2CMSNum (facilityName):
    if facilityName in ltc_name2cms_id:
        return ltc_name2cms_id[facilityName]
    else:
        return "No Match"

In [6]:
# 1 - Get a list of all IL json source files - DONE
# 2 - For Each JSON Source file:
#     2.1 - Turn it into a DataFrame - DONE
#     2.2 - Match to CMS Federal Provider Number- DONE
# 3 - Merge with CMS data associated, as closely as possible with that date

# NEXT ACTIONS:
# - Alllow for Renaming of fields (ie: State - Raw, State - Calc, CMS Raw, CMS Calc, ...)

# Prep 1 - Load Supporting Files (CMS et al)

In [7]:
# Load Facility Name to CMS ID json file
fac2CMS_file = 'IL_FacilityName_to_CMS_ID.json'
with open(fac2CMS_file) as f:
  ltc_name2cms_id = json.load(f) 

In [8]:
# Load CMS Dataset from CMS website
url_csv = 'https://data.cms.gov/api/views/s2uc-8wxp/rows.csv?accessType=DOWNLOAD&api_foundry=true'

response = urllib2.urlopen(url_csv)
cms_data = pd.read_csv(response, parse_dates=['Week Ending'], dtype={'Provider Name': str})
max_date = cms_data['Week Ending'].max()
reporting_date = str(max_date)[0:10]
print(reporting_date)
display(cms_data.head(5))
cms_data['Week Ending'] = pd.to_datetime(cms_data['Week Ending'])

/home/steve/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2020-11-22


,Week Ending,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Submitted Data,Passed Quality Assurance Check,Residents Weekly Admissions COVID-19,...,"Weekly Resident Confirmed COVID-19 Cases Per 1,000 Residents","Weekly Resident COVID-19 Deaths Per 1,000 Residents","Total Resident Confirmed COVID-19 Cases Per 1,000 Residents","Total Resident COVID-19 Deaths Per 1,000 Residents",Total Residents COVID-19 Deaths as a Percentage of Confirmed COVID-19 Cases,County,Three or More Confirmed COVID-19 Cases This Week,Initial Confirmed COVID-19 Case This Week,Geolocation,Reporting Interval
0,2020-05-24,105045,BRADEN RIVER REHABILITATION CENTER LLC,2010 MANATEE AVE E,BRADENTON,FL,34208,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manatee,NaN,NaN,POINT (-82.539097 27.496201000000003),Week 1 - May 24
1,2020-05-24,105384,CALUSA HARBOUR,2525 FIRST ST,FORT MYERS,FL,33901,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Lee,NaN,NaN,POINT (-81.864238 26.647471),Week 1 - May 24
2,2020-05-24,105453,KENSINGTON GARDENS REHAB AND NURSING CENTER,2055 PALMETTO ST,CLEARWATER,FL,33758,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pinellas,NaN,NaN,POINT (-82.751998 27.975286),Week 1 - May 24
3,2020-05-24,105460,NORTH FLORIDA REHABILITATION AND SPECIALTY CARE,6700 NW 10TH PLACE,GAINESVILLE,FL,32605,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Alachua,NaN,NaN,POINT (-82.413587 29.661501),Week 1 - May 24
4,2020-05-31,105478,ADVANCED CARE CENTER,401 FAIRWOOD AVE,CLEARWATER,FL,33759,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pinellas,NaN,NaN,POINT (-82.724356 27.970223),Week 2 - May 31


In [9]:
# Get CMS data for a particular state
myState = 'IL'
state_cms_data = cms_data[cms_data['Provider State'] == myState].add_prefix('CMS_')

### 1 - Get a list of all IL json source files
### 2 - For Each JSON Source file:
####     2.1 - Turn it into a DataFrame - DONE
####    2.2 - Match to CMS Federal Provider Number- DONE
####     2.1 - Merge with CMS data associated, as closely as possible with that date

In [10]:
# 1 - Get a list of all IL json source files
files = glob.glob('Source_data/IL_*LTC_data_Source.json')
facility_dfs = []
outbreak_dfs = []
# 2 - For Each JSON Source file:
#     2.1 - Turn it into a DataFrame - DONE
#    2.2 - Match to CMS Federal Provider Number- DONE
#     2.1 - Merge with CMS data associated, as closely as possible with that date
for file in files:
    [reporting_date, summary, outbreak_df, df_facilities, df_county] = process_json_IL(file, )

#     df_facilities.reset_index(inplace=True) # Needed because used group by to get facility level data ToDo: COnsider moving this code up
    df_facilities['STATE_county-facName']= df_facilities['STATE_County'].str.upper() + '-' + df_facilities['STATE_FacilityName'].str.upper()
    df_facilities['STATE_CMS_ProvNum'] = df_facilities['STATE_county-facName'].apply(lambda x: facility2CMSNum(x))
    facility_dfs.append(df_facilities)

    outbreak_df['STATE_county-facName'] = outbreak_df['STATE_County'].str.upper() + '-' + outbreak_df['STATE_FacilityName'].str.upper()
    outbreak_df['STATE_CMS_ProvNum'] = outbreak_df['STATE_county-facName'].apply(lambda x: facility2CMSNum(x))
    outbreak_dfs.append(outbreak_df)

Date: 2020-06-05
Cases: 18837
Deaths: 3053
Outbreaks: 554
Facilities: 554
Date: 2020-10-02
Cases: 30243
Deaths: 4697
Outbreaks: 967
Facilities: 891
Date: 2020-10-09
Cases: 30920
Deaths: 4792
Outbreaks: 1008
Facilities: 920
Date: 2020-10-23
Cases: 33440
Deaths: 5019
Outbreaks: 1151
Facilities: 1015
Date: 2020-11-06
Cases: 36683
Deaths: 5253
Outbreaks: 1309
Facilities: 1116
Date: 2020-06-19
Cases: 21390
Deaths: 3649
Outbreaks: 593
Facilities: 592
Date: 2020-11-20
Cases: 43222
Deaths: 5780
Outbreaks: 1532
Facilities: 1276
Date: 2020-09-18
Cases: 28941
Deaths: 4575
Outbreaks: 889
Facilities: 829
Date: 2020-06-12
Cases: 20550
Deaths: 3433
Outbreaks: 580
Facilities: 580
Date: 2020-10-30
Cases: 34278
Deaths: 5127
Outbreaks: 1209
Facilities: 1052
Date: 2020-08-28
Cases: 27126
Deaths: 4396
Outbreaks: 795
Facilities: 758
Date: 2020-08-21
Cases: 26355
Deaths: 4319
Outbreaks: 766
Facilities: 737
Date: 2020-07-10
Cases: 23324
Deaths: 3895
Outbreaks: 630
Facilities: 628
Date: 2020-09-11
Cases: 28189

In [11]:
all_facilities_df = pd.concat(facility_dfs)
all_facilities_df['STATE_ReportingDate']= pd.to_datetime(all_facilities_df['STATE_ReportingDate'])
all_facilities_df['STATE_MatchingDate'] = all_facilities_df['STATE_ReportingDate'].apply(lambda x: get_matching_cms_date(x))

all_outbreaks_df = pd.concat(outbreak_dfs)
all_outbreaks_df['STATE_reporting_date']= pd.to_datetime(all_outbreaks_df['STATE_reporting_date'])
all_outbreaks_df['STATE_MatchingDate'] = all_outbreaks_df['STATE_reporting_date'].apply(lambda x: get_matching_cms_date(x))

In [12]:
all_facilities_df.head()

,STATE_County,STATE_FacilityName,STATE_ReportingDate,STATE_confirmed_cases,STATE_deaths,STATE_CFR,STATE_outbreaks,STATE_facilities,STATE_county-facName,STATE_CMS_ProvNum,STATE_MatchingDate
0,Boone,Park Place of Belvidere,2020-06-05,5,0,0.000000,1,1,BOONE-PARK PLACE OF BELVIDERE,146071,2020-06-07
1,Boone,Symphony Northwoods,2020-06-05,87,13,0.149425,1,1,BOONE-SYMPHONY NORTHWOODS,No Match,2020-06-07
2,Champaign,Reflections Memory Care,2020-06-05,8,0,0.000000,1,1,CHAMPAIGN-REFLECTIONS MEMORY CARE,No Match,2020-06-07
3,Champaign,University Rehab Center,2020-06-05,5,0,0.000000,1,1,CHAMPAIGN-UNIVERSITY REHAB CENTER,145364,2020-06-07
4,Christian,Villas of Holly Brook,2020-06-05,2,0,0.000000,1,1,CHRISTIAN-VILLAS OF HOLLY BROOK,No Match,2020-06-07


In [13]:
all_facilities_df_match_list = []
for date in all_facilities_df['STATE_MatchingDate'].unique():
    state_cms_data_match = state_cms_data[state_cms_data['CMS_Week Ending'] == date]
    facilities_df_match = all_facilities_df[all_facilities_df['STATE_MatchingDate'] == date]
    facilities_df_match = add_cms_data_to_df(facilities_df_match, state_cms_data_match, 'STATE_CMS_ProvNum', 'CMS_Federal Provider Number')
    all_facilities_df_match_list.append(facilities_df_match)

In [14]:
all_facilities_df_match = pd.concat(all_facilities_df_match_list)

In [15]:
filename =  "Reporting_data/IL_" + reporting_date + "_Facilities_FullSummary_LTC_data_Source.csv"
all_facilities_df_match.to_csv(filename, index=False)

# filename =  "Reporting_data/IL_" + reporting_date + "_Outbreaks_FullSummary_LTC_data_Source.csv"
# all_outbreaks_df.to_csv(filename, index=False)

In [16]:
# CREATE File that matches CTP Format - FACILITY LEVEL
reporting_date ='2020-12-07'
filename =  "Reporting_data/IL_" + reporting_date + "_Facilities_FullSummary_LTC_data_Source.csv"
cols=['STATE_ReportingDate', 'CMS_Provider State', 'STATE_County', 'CMS_Provider City',
      'STATE_FacilityName', 'STATE_CMS_ProvNum','STATE_confirmed_cases', 'STATE_deaths',
      'STATE_MatchingDate', 'CMS_Week Ending', 'CMS_Federal Provider Number']
all_facilities_df_match[cols].to_csv(filename, index=False)

In [17]:
# Date Collected	State 	County	City	Facility Name	State Facility Type
# ReportingDate	Provider State	County_x	Provider City	FacilityName	FacilityName

State_to_CTP_Field_name_map = {}
xx = 'Date Collected	State 	County	City	Facility Name	State Facility Type	CTP Facility Categorization	State/Fed Regulated	State Facility ID	CMS Facility ID	Date outbreak opened	Date outreak closed	Outbreak Status	Resident Census	Resident Positives	Resident Probable Positives	Resident Deaths	Resident Probable Deaths	Staff Positive	Staff Probable Positives	Staff Deaths	Staff Probable Deaths	Resident/Staff Positives	Resident/Staff Probable Positives	Resident/Staff Deaths'
for item in xx.split('\t'):
    State_to_CTP_Field_name_map[item] = ''
    
State_to_CTP_Field_name_map['Date Collected'] = 'ReportingDate'
State_to_CTP_Field_name_map['State '] = 'Provider State'
State_to_CTP_Field_name_map['County'] = 'County_x'
State_to_CTP_Field_name_map['City'] = 'Provider City'
State_to_CTP_Field_name_map['Facility Name'] = 'FacilityName'
State_to_CTP_Field_name_map['Resident/Staff Positives'] = 'confirmed_cases'
State_to_CTP_Field_name_map['Resident/Staff Deaths'] = 'deaths'


In [18]:
all_facilities_df_match.columns

Index(['STATE_County', 'STATE_FacilityName', 'STATE_ReportingDate',
       'STATE_confirmed_cases', 'STATE_deaths', 'STATE_CFR', 'STATE_outbreaks',
       'STATE_facilities', 'STATE_county-facName', 'STATE_CMS_ProvNum',
       'STATE_MatchingDate', 'CMS_Week Ending', 'CMS_Federal Provider Number',
       'CMS_Provider Name', 'CMS_Provider Address', 'CMS_Provider City',
       'CMS_Provider State', 'CMS_County', 'CMS_Provider Zip Code',
       'CMS_Submitted Data', 'CMS_Residents Weekly Admissions COVID-19',
       'CMS_Residents Total Admissions COVID-19',
       'CMS_Residents Weekly Confirmed COVID-19',
       'CMS_Residents Total Confirmed COVID-19',
       'CMS_Residents Weekly Suspected COVID-19',
       'CMS_Residents Total Suspected COVID-19',
       'CMS_Residents Weekly All Deaths', 'CMS_Residents Total All Deaths',
       'CMS_Residents Weekly COVID-19 Deaths',
       'CMS_Residents Total COVID-19 Deaths', 'CMS_Number of All Beds',
       'CMS_Total Number of Occupied Beds',


In [20]:
STATE_County	STATE_FacilityName	STATE_ReportingDate	STATE_confirmed_cases	STATE_deaths	STATE_CFR	STATE_outbreaks	STATE_facilities	STATE_county-facName	STATE_CMS_ProvNum	STATE_MatchingDate	CMS_Week Ending


SyntaxError: invalid syntax (<ipython-input-20-852474c8a732>, line 1)

In [21]:
cols=['STATE_ReportingDate', 'CMS_Provider State', 'STATE_County', 'CMS_Provider City',
      'STATE_FacilityName', 'STATE_CMS_ProvNum','STATE_confirmed_cases', 'STATE_deaths',
      'STATE_MatchingDate', 'CMS_Week Ending', 'CMS_Federal Provider Number']
all_facilities_df_match[cols].head()

,STATE_ReportingDate,CMS_Provider State,STATE_County,CMS_Provider City,STATE_FacilityName,STATE_CMS_ProvNum,STATE_confirmed_cases,STATE_deaths,STATE_MatchingDate,CMS_Week Ending,CMS_Federal Provider Number
0,2020-06-05,IL,Boone,BELVIDERE,Park Place of Belvidere,146071,5,0,2020-06-07,2020-06-07,146071
1,2020-06-05,NaN,Boone,NaN,Symphony Northwoods,No Match,87,13,2020-06-07,NaT,NaN
2,2020-06-05,NaN,Champaign,NaN,Reflections Memory Care,No Match,8,0,2020-06-07,NaT,NaN
3,2020-06-05,IL,Champaign,URBANA,University Rehab Center,145364,5,0,2020-06-07,2020-06-07,145364
4,2020-06-05,NaN,Christian,NaN,Villas of Holly Brook,No Match,2,0,2020-06-07,NaT,NaN


In [22]:
#df_facilities_w_cms = add_cms_data_to_df(all_facilities_df, state_cms_data_latest, 'CMS_ProvNum', 'Federal Provider Number')


# 2 - Load Supporting Files (CMS et al)

In [23]:
# get_matching_cms_date('2020-10-26T00:00:00.000000000')

# Play Area

In [ ]:
all_outbreaks_df[all_outbreaks_df['STATE_FacilityName'] == 'Abington of Glenview'].sort_values(by='STATE_reporting_date').to_csv('Abington of Glenview.csv')

In [ ]:
all_outbreaks_df[all_outbreaks_df['STATE_FacilityName'] == 'Abington of Glenview'].sort_values(by='STATE_reporting_date')

In [ ]:
state_cms_data[state_cms_data['CMS_Federal Provider Number'] == '145683'].sort_values(by='CMS_Week Ending').to_csv('CMS - Abington of Glenview.csv')
state_cms_data[state_cms_data['CMS_Federal Provider Number'] == '145683'].sort_values(by='CMS_Week Ending')

In [ ]:
#ltc_name2cms_id

# OLD CODE

# Individual Facility Data